# Test Models

In [1]:
from jupyterthemes import jtplot
from skimage.util import montage as montage2d
from tensorflow.keras.optimizers import Adam
import UNets.MyImplementation.UNET_Adv7 as UNet
import UNets.Vanilla.UNet_Vanilla as UNetV
from MyFunctions.CreatePaths import CreatePaths
from MyFunctions.LoadImages import LoadImages
from MyFunctions.RunModels import RunModels

import SimpleITK as sitk
import tensorflow as tf
import pandas as pd
import numpy as np
import importlib
import os

import matplotlib.pyplot as plt
import matplotlib as mpl

from os import listdir
from os.path import isfile, join

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score


def showCTMontage(IMG, SIZE, SaveFig=False, save_fig_name=""):
    plt.figure(figsize=(SIZE, SIZE))
    plt.imshow(montage2d(IMG), alpha=1, cmap='gray')
    plt.axis('off')
    plt.show()
    if SaveFig:
        save_fig_path = os.path.join(os.curdir, "SavedFigures")
        plt.savefig(os.path.join(save_fig_path,
                                 save_fig_name+".pdf"), bbox_inches='tight')


def showCTMontageOverlay(IMG1, IMG2, SIZE=15, SaveFig=False, save_fig_name=""):
    fig, ax = plt.subplots(figsize=(SIZE, SIZE))
    try:
        ax.imshow(montage2d(IMG1), alpha=1, cmap='gray')
    except:
        print("Error: Img 1")
    try:
        ax.imshow(montage2d(IMG2, fill=0), alpha=0.5,
                  cmap=cmap, interpolation='none')
    except:
        print("Error: Img 2")
    plt.axis('off')

    if SaveFig:
        save_fig_path = os.path.join(os.curdir, "SavedFigures")
        plt.savefig(os.path.join(save_fig_path,
                                 save_fig_name+".pdf"), bbox_inches='tight')
    plt.show()


def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [12]:
# Change Class Variable below for different paths e.g. CT/PET
path = CreatePaths(DeviceFlag="PC", ScanTypeFlag="CT", TrainTestFlag="Train")

DATA_PATH = "D://MyMasters//Data//TestingData//"
IMGS_PATH = "D://MyMasters//Data//TestingData//imgs"
MSKS_PATH = "D://MyMasters//Data/TestingData//masks"

OUTPUT_PATH = "D://MyMasters//Output"

print("Image Path: "+"\t"+IMGS_PATH+"\n"+"Mask Path: " +
      "\t"+MSKS_PATH+"\n"+"Output Path: "+"\t"+OUTPUT_PATH)

ScanType = "CT"
Orientation = "Axial"
Orientations = ["Axial", "Sagittal", "Coronal"]

Orientations_3 = ["Axial", "Coronal"]

TestImgsStrings = [f for f in listdir(IMGS_PATH) if isfile(join(IMGS_PATH, f))]
TestMasksStrings = [f for f in listdir(
    MSKS_PATH) if isfile(join(MSKS_PATH, f))]

Image Path: 	D://MyMasters//Data//TestingData//imgs
Mask Path: 	D://MyMasters//Data/TestingData//masks
Output Path: 	D://MyMasters//Output


#### Load Refined Models

In [3]:
MyModel_Ax = UNet.UNet_Adv(input_shape=(256, 256, 1),
                           n_filters=32, activation=tf.keras.layers.LeakyReLU(), showSummary=False).CreateUnet()
MyModel_Ax.compile(optimizer=Adam(learning_rate=1e-4),
                   loss=dice_coef_loss, metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
MyModel_Ax.load_weights(
    "D://MyMasters//Code//Josias-Masters//TrainedModels//U-Net_7_3_Axial.h5")

MyModel_Sg = UNet.UNet_Adv(input_shape=(256, 256, 1),
                           n_filters=32, activation=tf.keras.layers.LeakyReLU(), showSummary=False).CreateUnet()
MyModel_Sg.compile(optimizer=Adam(learning_rate=1e-4),
                   loss=dice_coef_loss, metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
MyModel_Sg.load_weights(
    "D://MyMasters//Code//Josias-Masters//TrainedModels//U-Net_7_2_Sagittal.h5")

MyModel_Cor = UNet.UNet_Adv(input_shape=(256, 256, 1),
                            n_filters=32, activation=tf.keras.layers.LeakyReLU(), showSummary=False).CreateUnet()
MyModel_Cor.compile(optimizer=Adam(learning_rate=1e-4),
                    loss=dice_coef_loss, metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
MyModel_Cor.load_weights(
    "D://MyMasters//Code//Josias-Masters//TrainedModels//U-Net_7_3_Coronal.h5")
Model_ENS = [MyModel_Ax, MyModel_Sg, MyModel_Cor]
Model_ENS_3 = [MyModel_Ax, MyModel_Cor]

#### Ensemble Model

In [ ]:
MyModel_Ens = UNet.UNet_Adv(input_shape=(256, 256, 1),
                            n_filters=32, activation=tf.keras.layers.LeakyReLU(), showSummary=False).CreateUnet()
MyModel_Ens.compile(optimizer=Adam(learning_rate=1e-4),
                    loss=dice_coef_loss, metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
MyModel_Ens.load_weights(
    "D://MyMasters//Code//Josias-Masters//TrainedModels//U-Net_E_Axial.h5")

#### Load Vanilla Model

In [7]:
MyModel_V = UNetV.UNet_Vanilla(input_shape=(256, 256, 1)).CreateUnet()
MyModel_V.compile(optimizer=Adam(learning_rate=1e-4),
                  loss=dice_coef_loss, metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
MyModel_V.load_weights(
    "D://MyMasters//Code//Josias-Masters//TrainedModels//U-Net_V_Axial.h5")

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 256, 256, 64) 640         input_4[0][0]                    
__________________________________________________________________________________________________
activation_289 (Activation)     (None, 256, 256, 64) 0           conv2d_57[0][0]                  
__________________________________________________________________________________________________
batch_normalization_315 (BatchN (None, 256, 256, 64) 448         activation_289[0][0]             
____________________________________________________________________________________________

#### Calculate Dice Coeff Of Test Scans

In [ ]:
cnt = 0
ori = 0
#threshold = 0.5

Model_ENS = MyModel_V
Orientations = Orientations[0]

for model in Model_ENS:
    Orientation = Orientations[ori]
    print(Orientation)
    for TestImg in TestImgsStrings:

        TestImage, _, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                            ImgPath=IMGS_PATH, Orientation=Orientation).LoadScan()
        TestImage = np.expand_dims(TestImage, axis=3)
        TestMsk, Orig_Size, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                                           MskPath=MSKS_PATH, Orientation=Orientation).LoadScan()
        TestMsk = tf.cast(TestMsk, dtype='float32')
        TestMsk = np.expand_dims(TestMsk, axis=3)
        y_hat = model.predict(TestImage, batch_size=10, verbose=0)
        #y_hat[y_hat>threshold]  = 1
        #y_hat[y_hat<threshold]  = 0
        #y_hat[y_hat==threshold] = 1
        #print(TestImg+"\t Dice Coeff: " + str(dice_coef(TestMsk, y_hat).numpy()*100))
        print(dice_coef(TestMsk, y_hat).numpy()*100)
        cnt = cnt+1
    ori = ori+1
    cnt = 0

#### Calculate Precision

In [5]:
cnt = 0
ori = 0
threshold = 0.5

Model_ENS = Model_ENS_3
Orientations = Orientations_3

for model in Model_ENS:
    Orientation = Orientations[ori]
    print(Orientation)
    for TestImg in TestImgsStrings:

        TestImage, _, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                            ImgPath=IMGS_PATH, Orientation=Orientation).LoadScan()
        TestImage = np.expand_dims(TestImage, axis=3)
        TestMsk, Orig_Size, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                                           MskPath=MSKS_PATH, Orientation=Orientation).LoadScan()
        TestMsk = tf.cast(TestMsk, dtype='float32')
        TestMsk = np.expand_dims(TestMsk, axis=3)
        y_hat = model.predict(TestImage, batch_size=10, verbose=0)

        y_hat[y_hat > threshold] = 1
        y_hat[y_hat < threshold] = 0
        y_hat[y_hat == threshold] = 1

        #print(TestImg+"\t Dice Coeff: " + str(dice_coef(TestMsk, y_hat).numpy()*100))
        TestMsk = np.squeeze(TestMsk)
        y_hat = np.squeeze(y_hat)
        print(precision_score(TestMsk.flatten(), y_hat.flatten()) * 100)
        cnt = cnt+1
    ori = ori+1
    cnt = 0

Axial
91.65160486132751
89.26316081664376
89.84094840315572
93.78679639450117
99.12141059462081
97.36180465830918
99.59363673996975
86.90476190476191
83.89368312046945
92.37664720813805
88.27750720604314
87.44933497874673
88.10652450591935
90.19048536352253
Coronal
93.25677330640482
89.4771711704092
91.07864742195824
95.386195946577
98.37508776532133
97.24708496283384
99.29463873122161
90.48129063815074
86.05970856533585
93.24885339237704
89.52744805606063
88.03767067089062
90.16488816842859
93.20723890913662


#### Calculate Recall

In [6]:
cnt = 0
ori = 0
threshold = 0.5

Model_ENS = Model_ENS_3
Orientations = Orientations_3

for model in Model_ENS:
    Orientation = Orientations[ori]
    print(Orientation)
    for TestImg in TestImgsStrings:

        TestImage, _, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                            ImgPath=IMGS_PATH, Orientation=Orientation).LoadScan()
        TestImage = np.expand_dims(TestImage, axis=3)
        TestMsk, Orig_Size, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                                           MskPath=MSKS_PATH, Orientation=Orientation).LoadScan()
        TestMsk = tf.cast(TestMsk, dtype='float32')
        TestMsk = np.expand_dims(TestMsk, axis=3)
        y_hat = model.predict(TestImage, batch_size=10, verbose=0)

        y_hat[y_hat > threshold] = 1
        y_hat[y_hat < threshold] = 0
        y_hat[y_hat == threshold] = 1

        #print(TestImg+"\t Dice Coeff: " + str(dice_coef(TestMsk, y_hat).numpy()*100))
        TestMsk = np.squeeze(TestMsk)
        y_hat = np.squeeze(y_hat)
        print(recall_score(TestMsk.flatten(), y_hat.flatten()) * 100)
        cnt = cnt+1
    ori = ori+1
    cnt = 0

Axial
98.10097730734593
93.36846536881379
94.69073769398373
93.20131821547307
84.2848578997918
90.56404592922223
87.10616539575575
97.82680364605145
99.78991452874476
98.40082819119229
98.02441339425644
98.0400234889734
96.2078224478451
97.71703792895846
Coronal
96.51771717014486
92.15045802302456
92.50151926126847
88.96212308183678
82.1860677981397
87.43361538336607
84.14908080628953
95.59747306893817
99.42175171592612
96.3773446119897
96.72869346400901
97.31567765995715
95.09392601294302
95.25506722857716


## Run Ensemble Average Model
### Calculate Dice

In [ ]:
cnt = 0
for TestImg in TestImgsStrings:
    TestMsk, _, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                               MskPath=MSKS_PATH, Orientation=Orientations[0], ImgDepth=0, ImgSize=0).LoadScan()

    Arr, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                          ImgPath=IMGS_PATH, Orientations=Orientations).LoadScanEnsamble()

    y_hat = RunModels(OutPath=OUTPUT_PATH, ScanName="P"+TestImg, Scan="None", Scans=Arr, Scan_Size=Orig_Size,
                      Scan_Metadata=MetaData, Model=Model_ENS, Orientation="", Threshold=0.5, Orientations=Orientations).runModels(Limit=True, ReturnArray=True, Verbose=0)
    #y_hat = y_hat.numpy()
    y_hat = tf.cast(y_hat, dtype='float32')
    TestMsk = tf.cast(TestMsk, dtype='float32')
    print(dice_coef(TestMsk, y_hat).numpy()*100)
    cnt = cnt + 1

#### Precision

In [ ]:
cnt = 0
for TestImg in TestImgsStrings:
    TestMsk, _, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                               MskPath=MSKS_PATH, Orientation=Orientations[0], ImgDepth=0, ImgSize=0).LoadScan()

    Arr, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                          ImgPath=IMGS_PATH, Orientations=Orientations).LoadScanEnsamble()

    y_hat = RunModels(OutPath=OUTPUT_PATH, ScanName="P"+TestImg, Scan="None", Scans=Arr, Scan_Size=Orig_Size,
                      Scan_Metadata=MetaData, Model=Model_ENS, Orientation="", Threshold=0.5, Orientations=Orientations).runModels(Limit=True, ReturnArray=True, Verbose=0)

    y_hat = tf.cast(y_hat, dtype='float32')
    y_hat = y_hat.numpy()
    TestMsk = tf.cast(TestMsk, dtype='float32')
    TestMsk = TestMsk.numpy()
    print(precision_score(TestMsk.flatten(), y_hat.flatten())*100)
    cnt = cnt + 1

#### Recall

In [ ]:
cnt = 0
for TestImg in TestImgsStrings:
    TestMsk, _, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                               MskPath=MSKS_PATH, Orientation=Orientations[0], ImgDepth=0, ImgSize=0).LoadScan()

    Arr, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                          ImgPath=IMGS_PATH, Orientations=Orientations).LoadScanEnsamble()

    y_hat = RunModels(OutPath=OUTPUT_PATH, ScanName="P"+TestImg, Scan="None", Scans=Arr, Scan_Size=Orig_Size,
                      Scan_Metadata=MetaData, Model=Model_ENS, Orientation="", Threshold=0.5, Orientations=Orientations).runModels(Limit=True, ReturnArray=True, Verbose=0)
    #y_hat = y_hat.numpy()
    y_hat = tf.cast(y_hat, dtype='float32')
    y_hat = y_hat.numpy()
    TestMsk = tf.cast(TestMsk, dtype='float32')
    TestMsk = TestMsk.numpy()
    print(recall_score(TestMsk.flatten(), y_hat.flatten())*100)
    cnt = cnt + 1

## Run Ensemble Predict
#### Dice

In [ ]:
cnt = 0
for TestImg in TestImgsStrings:
    TestMsk, _, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                               MskPath=MSKS_PATH, Orientation=Orientations[0], ImgDepth=0, ImgSize=0).LoadScan()

    Arr, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                          ImgPath=IMGS_PATH, Orientations=Orientations).LoadScanEnsamble()

    y_hat = RunModels(OutPath=OUTPUT_PATH, ScanName="P"+TestImg, Scan="None", Scans=Arr, Scan_Size=Orig_Size,
                      Scan_Metadata=MetaData, Model=Model_ENS, Orientation="", EnsModel=MyModel_Ens, Threshold=0.5,
                      Orientations=Orientations).runEnsemble(Limit=True, ReturnArray=True, Verbose=0)
    y_hat = tf.cast(y_hat, dtype='float32')
    y_hat = y_hat.numpy()
    TestMsk = tf.cast(TestMsk, dtype='float32')
    TestMsk = TestMsk.numpy()
    print(dice_coef(TestMsk, y_hat).numpy()*100)
    cnt = cnt + 1

#### Precison

In [ ]:
cnt = 0
for TestImg in TestImgsStrings:
    TestMsk, _, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                               MskPath=MSKS_PATH, Orientation=Orientations[0], ImgDepth=0, ImgSize=0).LoadScan()

    Arr, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                          ImgPath=IMGS_PATH, Orientations=Orientations).LoadScanEnsamble()

    y_hat = RunModels(OutPath=OUTPUT_PATH, ScanName="P"+TestImg, Scan="None", Scans=Arr, Scan_Size=Orig_Size,
                      Scan_Metadata=MetaData, Model=Model_ENS, Orientation="", EnsModel=MyModel_Ens, Threshold=0.5,
                      Orientations=Orientations).runEnsemble(Limit=True, ReturnArray=True, Verbose=0)
    y_hat = tf.cast(y_hat, dtype='float32')
    y_hat = y_hat.numpy()
    TestMsk = tf.cast(TestMsk, dtype='float32')
    TestMsk = TestMsk.numpy()
    print(precision_score(TestMsk.flatten(), y_hat.flatten())*100)
    cnt = cnt + 1

#### Recall

In [ ]:
cnt = 0
for TestImg in TestImgsStrings:
    TestMsk, _, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                               MskPath=MSKS_PATH, Orientation=Orientations[0], ImgDepth=0, ImgSize=0).LoadScan()

    Arr, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                          ImgPath=IMGS_PATH, Orientations=Orientations).LoadScanEnsamble()

    y_hat = RunModels(OutPath=OUTPUT_PATH, ScanName="P"+TestImg, Scan="None", Scans=Arr, Scan_Size=Orig_Size,
                      Scan_Metadata=MetaData, Model=Model_ENS, Orientation="", EnsModel=MyModel_Ens, Threshold=0.5,
                      Orientations=Orientations).runEnsemble(Limit=True, ReturnArray=True, Verbose=0)
    y_hat = tf.cast(y_hat, dtype='float32')
    y_hat = y_hat.numpy()
    TestMsk = tf.cast(TestMsk, dtype='float32')
    TestMsk = TestMsk.numpy()
    print(recall_score(TestMsk.flatten(), y_hat.flatten())*100)
    cnt = cnt + 1

## Vanilla Model Tests
### Dice Coeff

In [14]:
model = MyModel_V
Orientation = Orientations[0]
print(Orientation)
cnt=0
for TestImg in TestImgsStrings:

    TestImage, _, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                        ImgPath=IMGS_PATH, Orientation=Orientation).LoadScan()
    TestImage = np.expand_dims(TestImage, axis=3)
    TestMsk, Orig_Size, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                                       MskPath=MSKS_PATH, Orientation=Orientation).LoadScan()
    TestMsk = tf.cast(TestMsk, dtype='float32')
    TestMsk = np.expand_dims(TestMsk, axis=3)
    y_hat = model.predict(TestImage, batch_size=10, verbose=0)
    #y_hat[y_hat>threshold]  = 1
    #y_hat[y_hat<threshold]  = 0
    #y_hat[y_hat==threshold] = 1
    #print(TestImg+"\t Dice Coeff: " + str(dice_coef(TestMsk, y_hat).numpy()*100))
    print(dice_coef(TestMsk, y_hat).numpy()*100)
    cnt = cnt+1

Axial
94.66692805290222
89.59042429924011
91.10270142555237
91.58167243003845
87.07539439201355
93.33435297012329
92.03633666038513
91.23505353927612
91.53825640678406
95.33188939094543
92.45620369911194
92.0780599117279
92.30145215988159
92.74770617485046


### Precison

In [15]:
cnt = 0
ori = 0
threshold = 0.5

model = MyModel_V
Orientation = Orientations[0]

print(Orientation)
for TestImg in TestImgsStrings:

    TestImage, _, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                        ImgPath=IMGS_PATH, Orientation=Orientation).LoadScan()
    TestImage = np.expand_dims(TestImage, axis=3)
    TestMsk, Orig_Size, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                                       MskPath=MSKS_PATH, Orientation=Orientation).LoadScan()
    TestMsk = tf.cast(TestMsk, dtype='float32')
    TestMsk = np.expand_dims(TestMsk, axis=3)
    y_hat = model.predict(TestImage, batch_size=10, verbose=0)

    y_hat[y_hat > threshold] = 1
    y_hat[y_hat < threshold] = 0
    y_hat[y_hat == threshold] = 1

    #print(TestImg+"\t Dice Coeff: " + str(dice_coef(TestMsk, y_hat).numpy()*100))
    TestMsk = np.squeeze(TestMsk)
    y_hat = np.squeeze(y_hat)
    print(precision_score(TestMsk.flatten(), y_hat.flatten()) * 100)
    cnt = cnt+1

Axial
92.31842099702034
89.08425434189752
89.67953135768437
93.9329649867212
98.95230828554317
97.57035173653861
99.66148704520397
87.2243080273701
84.67269174401564
93.97667686709123
88.65863128397312
87.93366326385022
89.21965827021315
89.85038232283702


#### Recall

In [16]:
cnt = 0
ori = 0
threshold = 0.5

model = MyModel_V
Orientation = Orientations[0]

print(Orientation)
for TestImg in TestImgsStrings:

    TestImage, _, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=TestImg,
                                        ImgPath=IMGS_PATH, Orientation=Orientation).LoadScan()
    TestImage = np.expand_dims(TestImage, axis=3)
    TestMsk, Orig_Size, _ = LoadImages(ScanType="CT", ScanClass="Mask", ScanName=TestMasksStrings[cnt],
                                       MskPath=MSKS_PATH, Orientation=Orientation).LoadScan()
    TestMsk = tf.cast(TestMsk, dtype='float32')
    TestMsk = np.expand_dims(TestMsk, axis=3)
    y_hat = model.predict(TestImage, batch_size=10, verbose=0)

    y_hat[y_hat > threshold] = 1
    y_hat[y_hat < threshold] = 0
    y_hat[y_hat == threshold] = 1

    #print(TestImg+"\t Dice Coeff: " + str(dice_coef(TestMsk, y_hat).numpy()*100))
    TestMsk = np.squeeze(TestMsk)
    y_hat = np.squeeze(y_hat)
    print(recall_score(TestMsk.flatten(), y_hat.flatten()) * 100)
    cnt = cnt+1

Axial
97.14368308113096
90.1020571113337
92.57859397927875
89.34684455800583
77.74418739565674
89.4499212317087
85.49314299225631
95.63578690379664
99.63183718718136
96.73627286720335
96.6080244869729
96.6362191361346
95.62517699625555
95.8346377684126


## Run

In [ ]:
ImgPath = "F:\\MyMasters\\Data\\TestingData\\imgs"
ScanName = "CB_108_CT_M1"

## Run Multiple Models On All Scans

In [ ]:
ImgPath = "F:\\MyMasters\\Data\\TrainingData\\imgs"
files = [f for f in listdir(ImgPath) if isfile(join(ImgPath, f))]
for file in files:
    Arr, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=file,
                                          ImgPath=ImgPath, Orientations=Orientations).LoadScanEnsamble()
    RunModels(OutPath=OUTPUT_PATH, ScanName="P"+file, Scan="None", Scans=Arr, Scan_Size=Orig_Size,
              Scan_Metadata=MetaData, Model=[MyModel_Ax, MyModel_Sg, MyModel_Cor], Orientation="", Orientations=Orientations).runModels(Limit=False)

## Run Multiple Models On One Scan

In [ ]:
ImgPath = "F:\\MyMasters\\Data\\TestingData\\imgs"
ScanName = "CB_080_CT_M1"
for i in range(0, 3):
    TestImage, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=ScanName+".nii.gz",
                                                ImgPath="F:\\MyMasters\\Data\\TestingData\\imgs", Orientation=Orientations[i]).LoadScan()
    RunModels(OutPath=OUTPUT_PATH, ScanName="P"+ScanName, Scan=TestImage, Scans="None", Scan_Size=Orig_Size,
              Scan_Metadata=MetaData, Model=Model_ENS[i], Orientation=Orientations[i]).runModel()

In [ ]:
ImgPath = "F:\\MyMasters\\Data\\TestingData\\imgs"
files = [f for f in listdir(ImgPath) if isfile(join(ImgPath, f))]
for file in files:
    TestImage, Orig_Size, MetaData = LoadImages(ScanType="CT", ScanClass="Image", ScanName=file,
                                                ImgPath=ImgPath, Orientation=Orientations[2]).LoadScan()
    RunModels(OutPath=OUTPUT_PATH, ScanName="P"+file, Scan=TestImage, Scans="", Scan_Size=Orig_Size,
              Scan_Metadata=MetaData, Model=MyModel_Cor, Orientation="Coronal", Orientations=Orientations).runModel()